# Imports

In [2]:
import os
import sys
import json
import xarray as xr
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

In [3]:
# go up until we are in the project base directory
base_dir = os.getcwd()
while base_dir.split('/')[-1] != 'provide':
    base_dir = os.path.normpath(os.path.join(base_dir, '..'))

# add paths for tools and data
things_to_add = ['general_tools', 'aggregation_tools', 'general_data_for_aggregation']
for thing in things_to_add:
    sys.path.append(os.path.join(base_dir, thing))

# import stuff we need
from general_tools import check_if_notebook, mkdir
from oggm_result_filepath_and_realisations import (gcms_mesmer, quantiles_mesmer,
    scenarios_mesmer, oggm_result_dir, provide_regions, raw_oggm_output_file)
from aggregation_preprocessing import open_grid_from_dict
from actual_aggregation import open_files_and_aggregate_on_map, aggregating_scenario, check_slurm_done

In [4]:
# Use this to conditionally execute tests/debugging
if check_if_notebook():
    is_notebook = True
else:
    is_notebook = False

# Define directories

In [5]:
resolution_dir = 'resolution_1_deg'

In [6]:
preprocess_country_dict_outpath = os.path.join(base_dir, 'countries', resolution_dir)
mkdir(preprocess_country_dict_outpath);

# Open data

In [7]:
with open(os.path.join(preprocess_country_dict_outpath, "preprocessed_country_grids.json"), 'r') as f:
    country_structure_dict = json.load(f)

# Code for running on cluster

## define experiments for multiprocessing

In [15]:
# create dict for sbatch --array=
slurm_arrays = {}

count = 1
for country in country_structure_dict.keys():
    for scenario in scenarios_mesmer:
        slurm_arrays[str(count)] = (country, scenario)
        count += 1

if is_notebook:
    print(slurm_arrays)

{'1': ('CHL', 'CurPol'), '2': ('CHL', 'GS'), '3': ('CHL', 'LD'), '4': ('CHL', 'ModAct'), '5': ('CHL', 'Ref'), '6': ('CHL', 'Ren'), '7': ('CHL', 'Neg'), '8': ('CHL', 'SP'), '9': ('CHL', 'ssp119'), '10': ('CHL', 'ssp534-over'), '11': ('CHN', 'CurPol'), '12': ('CHN', 'GS'), '13': ('CHN', 'LD'), '14': ('CHN', 'ModAct'), '15': ('CHN', 'Ref'), '16': ('CHN', 'Ren'), '17': ('CHN', 'Neg'), '18': ('CHN', 'SP'), '19': ('CHN', 'ssp119'), '20': ('CHN', 'ssp534-over'), '21': ('DEU', 'CurPol'), '22': ('DEU', 'GS'), '23': ('DEU', 'LD'), '24': ('DEU', 'ModAct'), '25': ('DEU', 'Ref'), '26': ('DEU', 'Ren'), '27': ('DEU', 'Neg'), '28': ('DEU', 'SP'), '29': ('DEU', 'ssp119'), '30': ('DEU', 'ssp534-over'), '31': ('NPL', 'CurPol'), '32': ('NPL', 'GS'), '33': ('NPL', 'LD'), '34': ('NPL', 'ModAct'), '35': ('NPL', 'Ref'), '36': ('NPL', 'Ren'), '37': ('NPL', 'Neg'), '38': ('NPL', 'SP'), '39': ('NPL', 'ssp119'), '40': ('NPL', 'ssp534-over'), '41': ('UZB', 'CurPol'), '42': ('UZB', 'GS'), '43': ('UZB', 'LD'), '44':

## run current experiment

In [ ]:
if not is_notebook:
    slurm_id = os.environ.get('ARRAY_ID', None)
    # convert slurm array to region and scenario
    country, scenario = slurm_arrays[slurm_id]

    # save results on cluster and copy at the end in run_slurm-file
    working_dir_cluster = os.environ.get('OGGM_WORKDIR', None)

    aggregated_data_outpath = os.path.join(
        working_dir_cluster, 'aggregated_data')
    mkdir(aggregated_data_outpath);

    aggregated_data_intermediate_outpath = os.path.join(
        preprocess_country_dict_outpath,
        'aggregated_data_intermediate')
    mkdir(aggregated_data_intermediate_outpath);

    aggregating_scenario(
            target_name=country,
            target_structure_dict=country_structure_dict,
            scenario=scenario,
            output_folder=aggregated_data_outpath,
            oggm_result_dir=oggm_result_dir,
            raw_oggm_output_file=raw_oggm_output_file,
            intermediate_data_folder=aggregated_data_intermediate_outpath,
            variables=['volume', 'area', 'thinning_rate'],
            risk_variables=['volume', 'area'],
            risk_thresholds=np.append(np.arange(10, 91, 10), [99]),  # in % melted of 2020, 10% means 10% of 2020 melted
            time_steps=np.arange(2015, 2101, 5),
            reset_files=False
        )

## check which experiments failed for rerunning

In [6]:
if is_notebook:
    check_slurm_done(435675)

Files without 'SLURM DONE':
sbatch --array=300,258,134,256,347,264,106,351,318,5,99,107,100,219,73,257,191,301,259,123,317,187,297,273,88,325,245,293,184,366,328,104,39,266,42,195,144,370,228,192,302,221,203,367,130,371,142,283,145,177,260,105 run_slurm_aggregation_workflow.sh


In [7]:
if is_notebook:
    check_slurm_done(436286)

Files without 'SLURM DONE':
sbatch --array=106,297,293,245,177 run_slurm_aggregation_workflow.sh


In [8]:
if is_notebook:
    check_slurm_done(436387)

All files contain 'SLURM DONE'.
